# Chatbot Flask API

In [ ]:
from flask import Flask, jsonify, request
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

app = Flask(__name__)

# Initialize model and tokenizer with the medium model for faster responses
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Add padding token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Predefined responses with links
predefined_responses = {
    "services": "At Easy Link Web & IT Solutions, we offer services like web design & development, digital marketing, Google Workspace, cloud hosting, and cybersecurity.",
    "web design": "Our web design services include both frontend and backend development. Learn more here: https://www.easylinkindia.com/web-design.php",
    "digital marketing": "We provide digital marketing solutions to boost your brand. Visit: https://www.easylinkindia.com/digital-marketing.php",
    "google workspace": "We are certified in Google Workspace deployment. More details: https://www.easylinkindia.com/workspace.php",
    "cloud hosting": "Our cloud hosting solutions are secure and scalable. Check here: https://www.easylinkindia.com/cloud-hosting-and-storage.php",
    "cybersecurity": "We offer comprehensive cybersecurity services. Find out more: https://www.easylinkindia.com/cybersecurity.php",
    "ecommerce solutions": "We provide end-to-end eCommerce solutions, from setup to optimization, ensuring a smooth user experience. More info: https://www.easylinkindia.com/ecommerce.php",
    "mobile apps": "We develop mobile and web-based applications tailored to meet specific business needs. Visit: https://www.easylinkindia.com/mobile-app.php",
    "seo services": "We provide SEO and SEM services to improve your website’s visibility and search rankings. Details here: https://www.easylinkindia.com/seo.php",
    "content marketing": "Our content marketing services help in creating engaging content to enhance brand presence. Learn more: https://www.easylinkindia.com/content-marketing.php",
    "contact": "Contact us at WorkFlo Greeta Towers, Industrial Estate, Perungudi, OMR Phase 1, Chennai, Tamil Nadu - 600096. Call +91 9585182141 or email info@easylinkindia.com."
}

# Mapping number inputs to keywords
number_to_keyword = {
    "1": "services",
    "2": "web design",
    "3": "google workspace",
    "4": "cloud hosting",
    "5": "contact",
    "6": "ecommerce solutions",
    "7": "mobile apps",
    "8": "seo services",
    "9": "content marketing",
    "10": "cybersecurity"
}

# Generate response function
def generate_response(user_input):
    # Check for predefined responses
    for keyword, response in predefined_responses.items():
        if keyword in user_input.lower():
            return response
    
    # Generate custom response
    inputs = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt").to(device)
    attention_mask = torch.ones(inputs.shape, device=device)  # Added attention mask to avoid warnings
    reply_ids = model.generate(
        inputs,
        attention_mask=attention_mask,  # Pass attention mask for reliable behavior
        max_length=150,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.4,
        top_k=50,
        do_sample=True
    )
    return tokenizer.decode(reply_ids[:, inputs.shape[-1]:][0], skip_special_tokens=True)

@app.route("/question", methods=["GET"])
def get_question():
    user_input = request.args.get("query")
    if user_input:
        if user_input.isdigit() and user_input in number_to_keyword:
            keyword = number_to_keyword[user_input]
            return jsonify({"response": predefined_responses[keyword]})
        else:
            return jsonify({"response": generate_response(user_input)})
    else:
        return jsonify({"error": "No query parameter provided"}), 400

@app.route("/test", methods=["GET"])
def test():
    return jsonify({"status": "Flask app is running"})

if __name__ == "__main__":
    app.run(port=4000, debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:4000
Press CTRL+C to quit
127.0.0.1 - - [11/Nov/2024 01:33:57] "GET / HTTP/1.1" 404 -
